<p style="font-weight: bold;text-align: center;font-size: 3em;color: #00338D">What can you do with pyESASky?</p>

This notebook shows examples of what a scientist may typically want to do within a Jupyter notebook using pyESASky, e.g:
* Download and inspect data from ESASky (images, spectra, catalogues, ...)
* Cross-match catalogues available through ESASky
* Upload your own data to ESASky, e.g.: 
  * Cross-matched tables
  * Vizier catalogues (astropy tables) 
  * User provided tables
  * Footprints
  * HiPS
* Interact with ESASky Functionalities:
  * Set the: FoV, HiPS
  * Go to a target or coordinates
  * Slide through a selection of HiPS
  * See the observations, catalogues and publications count in the current FoV
  * Load the data panel for observations, spectra and catalogues accessible in ESASky
  * JWST planning tool
  * Set the coordinates grid on and off

In [1]:
# Import the required python modules:
from pyesasky import ESASkyWidget
from pyesasky import Catalogue
from pyesasky import CatalogueDescriptor
from pyesasky import CooFrame
from pyesasky import ImgFormat
from pyesasky import FootprintSet
from pyesasky import FootprintSetDescriptor
from pyesasky import MetadataDescriptor
from pyesasky import MetadataType

import pandas as pd

In [2]:
# Instantiate the pyESASky instance
esasky = ESASkyWidget()

/Users/dbaines/anaconda3/envs/jlab3/lib/python3.7/site-packages/pyesasky/pyesasky.py:84: UserWarning: The ESASky server has been updated since your installation of pyESASky.

Some commands might malfunction. Please upgrade your installation if you experience any issue.

$pip install --upgrade pyesasky 

and if you're using Jupyter Lab: 

$jupyter labextension install pyesasky@latest
  + "$pip install --upgrade pyesasky \n\nand if you're using Jupyter Lab: \n\n$jupyter labextension install pyesasky@latest")


All of the functions are now documented. Use the IPython ? magic to read about the function. Use tab to complete function names etc:

In [3]:
?ESASkyWidget.goToRADec

Signature: ESASkyWidget.goToRADec(self, ra, dec)
Docstring:
Moves the center of the view to the specified coordinate 
in current coordinate system

Arguments:
ra -- float or string in sexagesimal or decimal format
dec -- float or string in sexagesimal or decimal format
File:      ~/anaconda3/envs/jlab3/lib/python3.7/site-packages/pyesasky/pyesasky.py
Type:      function


In [4]:
# Load the pyESASky instance
esasky

ESASkyWidget(layout=Layout(align_self='stretch', height='400px'))

### Set custom height of the view

In [5]:
esasky.setViewHeight('850px')

### Go to a target name resolved against SIMBAD

In [6]:
esasky.goToTargetName('COSMOS Field')

### Go to RA and Dec (decimal degrees or sexagesimal format)

In [7]:
esasky.goToRADec('150.11917', '+02.20583') #COSMOS Field

### Set the Field of View in degrees

In [8]:
esasky.setFoV(0.5)

### Change the background HiPS to Herschel PACS colour

In [ ]:
esasky.setHiPS('Herschel PACS RGB 70, 160 micron')
# type the following to see the available HiPS from ESASky:
# esasky.getAvailableHiPS()

<br />
<p style="font-weight: bold;text-align: center;font-size: 2.5em;color: #00338D">Download and inspect data from ESASky</p>

### Information about the available data in the current field of view can be retrieved. For example the number of sources per catalogue:

In [ ]:
catCount = esasky.getCataloguesCount()
catCount = pd.DataFrame.from_dict([catCount]).transpose()
catCount.columns = ['Count']
catCount

### Download sources from the XMM EPIC (3XMM-DR8) catalogue and Herschel PACS 100 micron Point Source Catalogue in the current field of view:

In [ ]:
# Firstly, plot the XMM-Newton Serendipitous Source catalogue in ESASky:
esasky.plotCatalogues('XMM-EPIC')

In [ ]:
# then parse the data with pandas
data=esasky.getResultPanelData()
xmmepic = pd.DataFrame.from_dict(data)
xmmepic

In [ ]:
# Plot the Herschel-HPPSC-100 catalogue in ESASky:
esasky.plotCatalogues('Herschel-HPPSC-100')

In [ ]:
# then parse the data with pandas
data=esasky.getResultPanelData()
HPPSC100 = pd.DataFrame.from_dict(data)
HPPSC100

### Download and check the postcard of an observation from Chandra in the current FoV

In [ ]:
# Firstly, plot the Chandra imaging observations in ESASky
esasky.plotObservations('Chandra')

In [ ]:
data=esasky.getResultPanelData()
chandra = pd.DataFrame.from_dict(data)
chandra

In [ ]:
# plot one of the Chandra observation postcards:
from IPython.display import Image
Image(chandra.iloc[5]['postcard_url'])

### Download data from XMM-Newton in the current FoV

In [ ]:
# Plot the XMM-Newton imaging observations in ESASky
esasky.plotObservations('XMM')

In [ ]:
data=esasky.getResultPanelData()
xmm_newton = pd.DataFrame.from_dict(data)
xmm_newton

### Display and get data from External Data Centres

In [ ]:
# View the available predefined external TAP services in ESASky,
# e.g. European Southern Observatory (ESO), MAST (STScI), Canadian 
# Astronomical Data Centre (CADC)
esasky.getAvailableTapServices()

In [ ]:
# View all the available external TAP missions:
esasky.getAllAvailableTapMissions()

In [ ]:
# Return the available data in the current field of from ESO (from the ESO TAP):
esasky.getTapServiceCount('ESO')

In [ ]:
# Plot the footprints and metadata in pyESASky for ESO VLT images in the current FoV 
esasky.plotTapService('eso-image-eso-vlt-u')

In [ ]:
data=esasky.getResultPanelData()
vlt = pd.DataFrame.from_dict(data)
vlt

<br />
<p style="font-weight: bold;text-align: center;font-size: 2.0em;color: #00338D">Cross-match catalogues using astropy</p>

### Cross-match the two catalogues using astropy.coordinates

In [ ]:
from astropy.coordinates import SkyCoord
from astropy.coordinates import ICRS, Galactic, FK4, FK5
from astropy import units as u

In [ ]:
ra1 = xmmepic['ra']
dec1 = xmmepic['dec']
ra2 = HPPSC100['ra']
dec2 = HPPSC100['dec']
c = SkyCoord(ra1, dec1, frame='icrs', unit='deg')
catalog = SkyCoord(ra2, dec2, frame='icrs', unit='deg')
idx, d2d, d3d = c.match_to_catalog_sky(catalog)

In [ ]:
# Set a maximum separation as 5 arcseconds
max_sep = 5.0 * u.arcsec
idx, d2d, d3d = c.match_to_catalog_3d(catalog)
sep_constraint = d2d < max_sep
c_matches = c[sep_constraint]
catalog_matches = catalog[idx[sep_constraint]]
catalog_matches
#catalog_matches

In [ ]:
# Convert the SkyCoord column to an astropy table 
# and add a Name column (for pyESASky)
from astropy.table import Table
import numpy as np
XMMEPIC_HPPSC100 = Table()
XMMEPIC_HPPSC100['ra'] = catalog_matches.ra
XMMEPIC_HPPSC100['dec'] = catalog_matches.dec
XMMEPIC_HPPSC100['Name'] = np.linspace(1, catalog_matches.size, catalog_matches.size)
XMMEPIC_HPPSC100

<br />
<p style="font-weight: bold;text-align: center;font-size: 2.5em;color: #00338D">Send your data to ESASky</p>

## Send the cross-matched table to ESASky

In [ ]:
# overlayCatalogueFromAstropyTable('<catName>', '<cooFrame>', <color>, <lineWidth>, <table>, '<raColname>', '<decColname>', '<mainIDColname>')
# where:
# - <catName> : name of the catalogue that will be used in pyESASky as label
# - <cooFRAME> : coordinate frame
# - <color> : HTML color. It could be a "Color name", "Hex color code" or "RGB color code"
# - <lineWidth> : width used to draw sources. From 1 to 10
# - <table> : name of the column containing a unique identifier for sources if any. None if not applicable
# - <raColname> : name of the RA column in degrees
# - <decColname> : name of the Dec column in degrees
# - <mainIDColname> : name of the column with the name of the source

esasky.overlayCatalogueFromAstropyTable('XMMEPIC_HPPSC100', 'J2000', '#ffff00', 10, XMMEPIC_HPPSC100, 'ra','dec','Name')

## Send a catalogue in VizieR to ESASky

In [ ]:
# Import the VizieR astroquery module
from astroquery.vizier import Vizier

In [ ]:
# Query the Véron-Cetty+ 2010 catalogue, VII/258/vv10, for the 
# Cosmos field with search radius 0.5 degrees
result = Vizier.query_region("Cosmos Field", 
                             radius=0.5*u.deg, catalog='VII/258/vv10')
print(result)

In [ ]:
Veron = result['VII/258/vv10']
Veron

In [ ]:
#transform the RA and Dec to degrees and add two new columns
c = SkyCoord(Veron['RAJ2000'], Veron['DEJ2000'], unit=(u.hourangle, u.deg))
Veron['RAJ2000'] = c.ra
Veron['DEJ2000'] = c.dec
Veron

In [ ]:
# Send the catalogue to ESASky
esasky.overlayCatalogueFromAstropyTable('Veron', 'J2000', '#33c9cc', 5, Veron, 'RAJ2000', 'DEJ2000','Name')

You can also upload your own tables from files or create your own tables and send them to ESASky.

## Send Footprints to ESASky

Go to another region in the sky, set the Fov and change the HiPS to Spitzer

In [ ]:
esasky.closeAllResultPanelTabs()
esasky.goToTargetName('[SPK2012] MWP1G300134-001035')
esasky.setFoV(2.3)
esasky.setHiPS("Spitzer cold SEIP IRAC-1-3-4 RGB bright")
#esasky.setHiPS("Spitzer", "http://cdn.skies.esac.esa.int/Spitzer/IRAC134_RGB_bright/")

In [ ]:
footprintDesc = FootprintSetDescriptor('test footprints', 'red', 5, 'id', 'id', 'stcs', 'ra', 'dec', [])

In [ ]:
esasky.overlayFootprintsFromCSV('./testfootprint.csv', ',', footprintDesc)

In [ ]:
esasky.closeResultPanelTab()

## Send your own HiPS to ESASky

This will start a tornado server opening up a port allowing web browser access to this path and all folders/files underneath

In [ ]:
esasky.goToRADec('266.41683', '-29.00781')
esasky.setFoV(90)
esasky.removeHiPS('User')
#Change the below directory to your own HiPS
esasky.addLocalHiPS('/Users/dbaines/Downloads/LFI_SkyMap_030_1024_R3.00_full_HiPS/')
esasky.openSkyPanel()

In [ ]:
esasky.setHiPSColorPalette('PLANCK')

In [ ]:
#Stop the tornado server:
esasky.httpserver.stop()

## Send HiPS from any URL to ESASky

There are many additional HiPS available from many HiPS nodes, all listed here in the Aladin HiPS list: https://aladin.u-strasbg.fr/hips/list 

Select which HiPS you'd like to load in the list, copy the HiPS URL from the HiPS ID column in the HiPS sky maps section and paste the URL into the esasky.addHiPS command. Examples:

In [ ]:
esasky.addHiPS("Skymapper", "https://alasky.u-strasbg.fr/Skymapper/skymapper_color/")
esasky.addHiPS("PanSTARRS DR1 color-i-r-g", "https://alasky.u-strasbg.fr/Pan-STARRS/DR1/color-i-r-g/")
esasky.addHiPS("DECaLS DR5 color", "https://alasky.u-strasbg.fr/DECaLS/DR5/color/")
esasky.addHiPS("ROSAT", "https://alasky.u-strasbg.fr/RASS/")

<br />
<p style="font-weight: bold;text-align: center;font-size: 2.5em;color: #00338D">Interacting with other ESASky Functionalities</p>

## Perform Cone, Box and Polygon Searches

There are a couple of ways to perform a cone search with pyESASky. 

#### Method 1) use the coneSearchCatalogues, Observations or Spectra commands:

In [ ]:
esasky.goToRADec('15 0 0', '-59')
esasky.setFoV(5)

In [ ]:
?esasky.coneSearchCatalogues

In [ ]:
esasky.coneSearchCatalogues('Herschel-HPPSC-160', '225.0', '-59', 1 )

In [ ]:
esasky.coneSearchObservations('Chandra', '225.0', '-59', 1 )

In [ ]:
esasky.coneSearchSpectra('ISO-IR', '225.0', '-59', 1 )

In [ ]:
esasky.closeAllResultPanelTabs()

#### Method 2) use the setConeSearchArea command:

Define the Ra, Dec and radius, then data returned will only fall within this region. 

In [ ]:
ra = 225.0
dec = -59.0
radius = 1.0 # in decimal degrees
esasky.setConeSearchArea(ra, dec, radius)

In [ ]:
esasky.plotCatalogues('Herschel-HPPSC-160')
esasky.plotObservations('Chandra')
esasky.plotSpectra('ISO-IR')

#### To perform a box or polygon search, use the setPolygonSearchArea(STCS) command. 

STCS strings are expected to follow the format of 'BOX|POLYGON ICRS RA(1) Dec(1) ... RA(n) Dec(n)'.

In [ ]:
esasky.setPolygonSearchArea('BOX ICRS 226.8297 -58.0490 223.2086 -58.0496 223.1085 -59.9203 226.9319 -59.9197')

In [ ]:
esasky.setPolygonSearchArea('POLYGON ICRS 226.8682 -58.2503 225.3984 -60.1154 222.9764 -57.9569')

In [ ]:
esasky.clearSearchArea()

The Search tool panel can also be opened and closed with the following:

In [ ]:
esasky.showSearchToolPanel()

In [ ]:
esasky.closeSearchToolPanel()

In [ ]:
esasky.clearSearchArea()

In [ ]:
esasky.closeAllResultPanelTabs()

## Sliding through different HiPS

In [ ]:
# Go to Seyfert 2 Galaxy Centaurus A
esasky.removeHiPS('User')
esasky.goToRADec(201.365063, -43.019113)
esasky.setFoV(0.2)

Add HiPS to the sky panel

In [ ]:
esasky.addHiPS("Chandra RGB")
esasky.addHiPS("XMM-Newton EPIC color")
esasky.addHiPS("DSS2 Color")
esasky.addHiPS("2MASS color JHK")
esasky.addHiPS("Spitzer cold SEIP IRAC2 faint")
esasky.addHiPS("Herschel PACS RGB 70, 160 micron")

Animations can be made programmatically using this slider function

In [ ]:
import time
for i in range(500):
    esasky.setHiPSSliderValue(i/100.0)
    time.sleep(0.02)

We can even create a slider in the python interface to control the HiPS

In [ ]:
import asyncio
nSkies = esasky.getNumberOfSkyRows()
def wait_for_change(widget, value):
    future = asyncio.Future()
    def getvalue(change):
        future.set_result(change.new)
        widget.unobserve(getvalue, value)
    widget.observe(getvalue, value)
    return future

from ipywidgets import FloatSlider
slider = FloatSlider(min=0.0, max=nSkies-1, step=0.01,  continuous_update=True)

async def f():
    while True:
        x = await wait_for_change(slider, 'value')
        esasky.setHiPSSliderValue(x)
asyncio.ensure_future(f())

slider

## Interacting with the JWST Planning Tool

### Open the JWST Planning Tool and add an instrument at specific coordinates

In [ ]:
esasky.closeSkyPanel() #first, close the sky panel
esasky.addJwstWithCoordinates("NIRSpec", "NRS_FULL_MSA", False, "201.365063", "-43.019113", "5")

### Add an instrument at the centre of the screen and include all instruments in the focal plane

In [ ]:
esasky.setFoV(0.5)
esasky.addJwst("NIRCam", "NRCALL_FULL", True)

In [ ]:
esasky.addJwstWithCoordinates?

In [ ]:
esasky.closeJwstPanel()

In [ ]:
esasky.openJwstPanel()

In [ ]:
esasky.clearJwstAll()
esasky.closeJwstPanel()

## Coordinates Grid

Switch on or off the coordinates grid:

In [ ]:
esasky.goToRADec(350.850, +58.815) #Cas A
esasky.setFoV(0.2)

In [ ]:
esasky.setHiPS('Chandra RGB')
esasky.showCoordinateGrid(True)

In [ ]:
esasky.showCoordinateGrid(False)

## Healpix MOCs

Display your own multi-order coverage (MOC) maps on the sky using IVOA string or JSON notation

In [ ]:
esasky.setHiPS('DSS2 color')
esasky.goToRADec(43.376, +05.8587) #Cas A
esasky.setFoV(150)
esasky.overlayMOC('6/2-15 23 554 7/22 50-75', 'myMoc', 'red', 0.2)

In [ ]:
esasky.overlayMOC('{"3":[1,2,3,4,5], "4":[26,27]}', 'myMoc2', '#FF1244', 1)

In [ ]:
esasky.removeMOC('myMoc')
esasky.removeMOC('myMoc2')

## Multi-Messenger Events

Return and visualise the multi-messenger events in ESASky.

#### Gravitational Wave (GW) Events:

Return the IDs of all available GW events in ESASky

In [ ]:
gwdata=esasky.getGWIds() 
GWs = pd.DataFrame.from_dict(gwdata)
GWs

Return the metadata of all available GW Events in ESASky

In [ ]:
pd.set_option('max_colwidth', 400)
gwdata=esasky.getGWData()
GWs = pd.DataFrame.from_dict(gwdata)
GWs

Open the GWs Multi-Messenger Events in ESASky and display a GW Event

In [ ]:
esasky.openGWPanel()
esasky.showGWEvent('S200316bj') 

#### Neutrino Events:

Return the metadata of all available Neutrino events in ESASky

In [ ]:
ndata=esasky.getNeutrinoEventData()
neutrinos = pd.DataFrame.from_dict(ndata)
neutrinos

Open the neutrino event panel

In [ ]:
esasky.openNeutrinoPanel()

Close the Multi-Messenger panel in ESASky

In [ ]:
esasky.closeAlertPanel() 

### Save and restore an ESASky session

Save the current ESASky session as a JSON file object with all settings, HiPS stack, datapanels etc.

In [ ]:
esasky.saveSession(fileName='ESASkySession.json')

Change ESASky settings and then restore the previous session

In [ ]:
esasky.showCoordinateGrid(False)
esasky.setHiPS('DSS2 color')
esasky.goToTargetName('M101')

In [ ]:
esasky.restoreSessionFromFile('ESASkySession.json')